<a href="https://colab.research.google.com/github/kyileiaye2021/biomedgpt_hf/blob/main/biomedgpt_hf_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### use the pre-trained biomedgpt from huggingface -- PanaceaAI/BiomedGPT-Base-Pretrained

In [1]:
# download the same-size pretrained HF variant of the finetuned variant that you want to convert from.
model_name = "BiomedGPT-Base-Pretrained"
!git clone https://huggingface.co/PanaceaAI/{model_name}

Cloning into 'BiomedGPT-Base-Pretrained'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 10 (delta 0), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (10/10), 536.86 KiB | 4.59 MiB/s, done.


In [2]:
!git clone --single-branch --branch feature/add_transformers https://github.com/OFA-Sys/OFA.git

!pip install OFA/transformers

Cloning into 'OFA'...
remote: Enumerating objects: 5745, done.
remote: Counting objects: 100% (932/932), done.
remote: Compressing objects: 100% (256/256), done.
remote: Total 5745 (delta 710), reused 676 (delta 676), pack-reused 4813 (from 1)
Receiving objects: 100% (5745/5745), 97.78 MiB | 22.21 MiB/s, done.
Resolving deltas: 100% (2243/2243), done.
Processing ./OFA/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 5.2 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.18.0.dev0-py3-none-any.whl size=3916842 sha256=28e27a865ab49d6bb4872cf54b16aab86a43d5c1e0b768cddf51a7b05ed0e85b
  Stored in directory: /tmp/pip-ephem-wheel-cache-jegay4xc/wheels/0b/bc/ea/00b6b8998c20c4fe55affe6062a2cddda80308ef9bd5d5877c
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: tra

In [ ]:
from transformers import OFATokenizer, OFAModel
tokenizer = OFATokenizer.from_pretrained(f"./{model_name}")
model = OFAModel.from_pretrained(f"./{model_name}")


./BiomedGPT-Base-Pretrained
<super: <class 'OFATokenizer'>, <OFATokenizer object>>


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1440: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [ ]:
import re

import torch
from PIL import Image
from torchvision import transforms


mean, std = [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]
resolution = 480

patch_resize_transform = transforms.Compose([
        lambda image: image.convert("RGB"),
        transforms.Resize((resolution, resolution), interpolation=Image.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])


In [ ]:
!gdown --id 1kojeqtJSx3MzdY3jd--XzeJrdyiFuzC8 -O example.jpg

img = Image.open('example.jpg')

txt = "What modality is used to take this image?"
inputs = tokenizer([txt], return_tensors="pt").input_ids
patch_img = patch_resize_transform(img).unsqueeze(0)

gen = model.generate(inputs, patch_images=patch_img, num_beams=5, no_repeat_ngram_size=3, max_length=16)
results = tokenizer.batch_decode(gen, skip_special_tokens=True)

result = results[0]
result = re.sub(r'[^\w\s]', '', result).strip()

result

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1kojeqtJSx3MzdY3jd--XzeJrdyiFuzC8
To: /content/OFA-base/OFA-base/example.jpg
100% 14.6k/14.6k [00:00<00:00, 22.1MB/s]


'MRI'